In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objects as go
import plotly.express as px


import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/newsarticle/news.article.json


In [2]:
import json

In [6]:
df = pd.read_json(path_or_buf='/kaggle/input/newsarticle/news.article.json')
df.tail()

,articleBody,dateModified,scrapedDate,source,title
37416,Lebanese media are reporting renewed IDF strik...,NaN,{'$date': '2024-03-31T03:10:17.646Z'},https://www.timesofisrael.com/,Lebanese media reports renewed IDF strikes in ...
37417,Amid escalating tensions and concerns over pot...,{'$date': '2024-03-31T03:03:52.000Z'},{'$date': '2024-03-31T03:09:38.331Z'},https://www.deccanchronicle.com/,"US approves additional bombs, warplanes sales ..."
37418,"Haaretz.com, the online English edition of Haa...",{'$date': '2024-03-28T00:00:00.000Z'},{'$date': '2024-03-31T03:09:46.683Z'},https://www.haaretz.com/,Israel Publishes Video of Islamic Jihad Terror...
37419,"The UN secretary general, António Guterres, ha...",{'$date': '2024-03-31T00:00:00.000Z'},{'$date': '2024-03-31T03:09:50.586Z'},https://www.theguardian.com/,United Nations secretary general condemns expl...
37420,Aam Aadmi Party (AAP) leader Raghav Chadha rec...,{'$date': '2024-03-26T16:01:17.000Z'},{'$date': '2024-03-31T03:13:43.646Z'},https://indianexpress.com/,Raghav Chadha stirs a row as he meets this UK ...


In [7]:
print ("Rows     : " ,df.shape[0])
print ("Columns  : " ,df.shape[1])
print ("\nFeatures : \n" ,df.columns.tolist())
print ("\nMissing values :  ", df.isnull().sum().values.sum())

Rows     :  37421
Columns  :  5

Features : 
 ['articleBody', 'dateModified', 'scrapedDate', 'source', 'title']

Missing values :   13271


In [8]:
import json
import pandas as pd

# Reading the JSON file
with open('/kaggle/input/newsarticle/news.article.json', 'r') as file:
    articles = json.load(file)

# Extracting fields with fallback for missing keys
articleBody = [article.get('articleBody', None) for article in articles]
dateModified = [article.get('dateModified', None) for article in articles]
scrapedDate = [article.get('scrapedDate', None) for article in articles]
source = [article.get('source', None) for article in articles]
title = [article.get('title', None) for article in articles]

# Creating a DataFrame
df = pd.DataFrame({
    'ArticleBody': articleBody,
    'DateModified': dateModified,
    'ScrapedDate': scrapedDate,
    'Source': source,
    'Title': title
})

print(df.head())

                                         ArticleBody  \
0  Sanjay Raut, a member of the Shiv Sena (UBT) p...   
1  Kozhikode (Kerala) [India], October 27 (ANI): ...   
2  Mumbai, Oct 24 (PTI) Maharashtra Chief Ministe...   
3  Sensex, Nifty rebound over 1 pc after six sess...   
4  October 26, 2023 08:15 pm | Updated 08:38 pm I...   

                            DateModified  \
0  {'$date': '2023-10-25T06:35:50.000Z'}   
1                                   None   
2  {'$date': '2023-10-25T02:14:27.000Z'}   
3                                   None   
4  {'$date': '2023-10-26T14:45:24.000Z'}   

                             ScrapedDate                             Source  \
0  {'$date': '2023-10-27T13:12:18.339Z'}      https://www.thehansindia.com/   
1  {'$date': '2023-10-27T13:12:45.595Z'}            https://www.aninews.in/   
2  {'$date': '2023-10-27T13:12:18.339Z'}            https://thefederal.com/   
3  {'$date': '2023-10-27T13:12:41.618Z'}  https://english.varthabharati.in/   
4  

In [13]:
import spacy
from tqdm import tqdm
nlp= spacy.load('en_core_web_lg')
sent_vecs={}
docs=[]
for title in tqdm(df.Title):
    doc= nlp(title)
    docs.append(doc)
    sent_vecs.update({title: doc.vector})
sentences= list(sent_vecs.keys())
vectors= list(sent_vecs.values())

100%|██████████| 37421/37421 [05:14<00:00, 118.93it/s]


In [14]:
for sent, vec in list(sent_vecs.items())[:5]:
    print(f"Title: {sent}\nVector: {vec}\n")

Title: Shiv Sena MP Sanjay Raut Responds To 'Hamas' Remark In Fiery Exchange
Vector: [ 4.06419337e-01  6.60593450e-01  1.89651620e+00 -1.85392964e+00
  2.87645149e+00 -2.56565404e+00  6.82047784e-01 -3.57134700e+00
  6.32708427e-03 -1.68030572e+00 -1.24845791e+00  2.50371527e-02
 -1.15845811e+00  1.61273003e+00 -4.37262863e-01  1.96150970e+00
 -6.55937493e-01 -1.14559150e+00  2.00587675e-01  2.58024740e+00
 -2.28731132e+00 -1.91274196e-01 -1.38889313e-01  2.12943349e-02
  1.65574658e+00  6.34302914e-01  4.55127984e-01  1.84821379e+00
 -2.90556341e-01 -7.88770914e-02  1.18335342e+00  1.24759793e+00
  5.01572192e-01  1.97117007e+00 -1.80905795e+00 -1.28949359e-01
 -6.65521145e-01 -1.23614144e+00 -2.22306728e+00 -1.89655495e+00
  7.47887194e-01 -3.19590002e-01 -4.00626481e-01  1.63259315e+00
 -3.04730105e+00  4.90129322e-01 -1.06907070e+00 -1.91343334e-02
 -1.70004690e+00  1.86709499e+00 -6.75457895e-01 -1.57859004e+00
  2.33688164e+00 -1.33988369e+00 -1.77589285e+00  3.64437280e-03
 -2.0

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
x=np.array(vectors)
n_classes={}
for i in tqdm(np.arange(0.001, 1, 0.002)):
    dbscan= DBSCAN(eps=i, min_samples=2, metric='cosine').fit(x)
    n_classes.update({i: len(pd.Series(dbscan.labels_).value_counts())})
dbscan= DBSCAN(eps=0.08, min_samples=2, metric='cosine').fit(x)

  9%|▉         | 46/500 [06:12<1:01:24,  8.12s/it]